In [1]:
import dmd_with_power_control_single_task as DMD
import imageio
import numpy as np
import time
from glob import glob
import os
import importlib
importlib.reload(DMD)
import napari as nap

In [2]:
XP_FOLDER = r'D:\Data\DAN-trace-1CS' # 
TARGET_FOLDER = XP_FOLDER + r'\2_1__mVenus_20240709_152423' # D:\Data\OR83b_ppk72__20xpair_half\14_1__mVenus_20240625_150402
GENERAL_PARAMETERS = r'C:\\Users\\Light Sheet User\\Desktop\\DMD_parameters.pickled'

mirrors = DMD.dmd(XP_FOLDER, TARGET_FOLDER, GENERAL_PARAMETERS)
# Prints basic information about the DMD.
mirrors.idle()

Loading library: C:/Program Files/ALP-4.3/ALP-4.3 API/x64/alp4395.dll
DMD found, resolution = 1920 x 1200.
DMD patterns folder already exists.


c:\Users\Light Sheet User\Documents\GitHub\DMD\dmd_with_power_control_single_task.py:42: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\\\Users\\\\Light Sheet User\\\\Desktop\\\\DMD_parameters.pickled'>
  self.parameters = pickle.load(open(general_parameters_folder, 'rb'))


SELECT ROIs

In [3]:
mirrors.select_ROIs()

IMPORT the ROIs for usage

In [4]:
mirrors.import_N_rois(exp='2_1')

Found 3 ROI(s)


Learning Protocol - 20 Training Trials, 10s ISI Gap, 15s ITI gap

In [5]:
masks_path = r'D:\Data\OR83b_ppk72__20xpair_half\excitation_patterns\15_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\15_1__DAN_20240625_170309' # D:\Data\OR83b_ppk72__20xpair_half\14_1__DAN_20240625_151102
protocol_name = r"/15_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

on_time = 10
off_time = 30
train_off_time = 10 # changed from 10
rep_dead_time = 15

for i in range(2):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(60)

for i in range(19):
    mirrors.project(odpun, power); time.sleep(on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(rep_dead_time)

mirrors.project(odpun, power); time.sleep(on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(60)

for i in range(3):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", 
                         "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", 
                         "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", 
                         "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", 
                         "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", "1pun", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         ], \
                durations=[on_time, off_time, on_time, off_time, 
                           on_time, off_time, on_time, off_time,
                           on_time, off_time,on_time, 60,
                           on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                           on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time,on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                           on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time, on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time,on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time,on_time, train_off_time, on_time, rep_dead_time,
                            on_time, train_off_time, on_time, rep_dead_time,on_time, train_off_time, on_time, 60,
                            on_time, off_time, on_time, off_time, 
                           on_time, off_time, on_time, off_time,
                           on_time, off_time,on_time, off_time,
                           ]
                ) # 24.25 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_24624\4152795515.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_24624\4152795515.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_24624\4152795515.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input

Learning Protocol - Trace Conditioning - 20 Training Trials, 10s long stimulus, 1s ISI Gap, 15s ITI gap

In [5]:
masks_path = r'D:\Data\OR83b_ppk72__20xpair_trace_subregion\excitation_patterns\6_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\6_1__DAN_20240624_153056' # D:\Data\g12\
protocol_name = r"/6_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

for i in range(2):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(19):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(rep_dead_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(3):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "pun", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         ], \
                durations=[odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, 60, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, train_off_time, odor_on_time, 60, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           odor_on_time, off_time, odor_on_time, off_time, 
                           ]
                ) # 30.8 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_21740\1044106568.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_21740\1044106568.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_21740\1044106568.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input

Learning Protocol - Delay Conditioning - 20 Training Training Trials, 20s Stimulus Odor, 10s Delay, 10s ISI Gap, 15s ITI Gap

In [5]:
masks_path = r'D:\Data\OR83b_ppk72__20xpair_delay\excitation_patterns\6_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\6_1__DAN_20240702_165146' # D:\Data\OR83b_ppk72__20xpair_delay\1_1__DAN_20240630_155746
protocol_name = r"/6_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
od1pun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

on_time = 20
on_time_split = 10
off_time = 30
train_off_time = 10 # changed from 10
rep_dead_time = 15

for i in range(2):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(60)

for i in range(19):
    mirrors.project(img1, power); time.sleep(on_time_split); mirrors.idle()
    mirrors.project(od1pun, power); time.sleep(on_time_split); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(rep_dead_time)

mirrors.project(img1, power); time.sleep(on_time_split); mirrors.idle()
mirrors.project(od1pun, power); time.sleep(on_time_split); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(60)

for i in range(3):
    mirrors.project(img1, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)
    mirrors.project(img2, power); time.sleep(on_time); mirrors.idle(); time.sleep(off_time)


# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "1pun", "dark", "2", "dark", 
                         "1", "dark", "2", "dark", "1", "dark", "2", "dark", "1", "dark", "2", "dark",
                         ], \
                durations=[on_time, off_time, on_time, off_time, 
                           on_time, off_time, on_time, off_time,
                           on_time, off_time, on_time, 60,
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, rep_dead_time, 
                           on_time_split, on_time_split, train_off_time, on_time, 60, 
                           on_time, off_time, on_time, off_time, 
                           on_time, off_time, on_time, off_time,
                           on_time, off_time, on_time, off_time,
                           ]
                ) # 32.9 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_17904\1257816226.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_17904\1257816226.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_17904\1257816226.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input

Learning Protocol - Trace Conditiong - 20 Training Trials, 15 s stimulus, Only 1 US

First Phase - Learning

In [5]:
masks_path = r'D:\Data\DAN-trace-1CS\excitation_patterns\2_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\2_1__mVenus_20240709_152423' # D:\Data\g12\
protocol_name = r"/2_1"
images_folder = mirrors.xp_folder + current_acquisition


img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

for i in range(2):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(19):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
    mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(rep_dead_time)

mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time)
mirrors.project(img3, power); time.sleep(pun_on_time); mirrors.idle(); time.sleep(60)

for i in range(3):
    mirrors.project(img1, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["1", "dark", "1", "dark", "1", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "pun", "dark", 
                         "1", "dark", "1", "dark",  "1", "dark",
                         ], \
                durations=[odor_on_time, off_time, 
                           odor_on_time, off_time, 
                           odor_on_time, 60, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, rep_dead_time, 
                           odor_on_time, train_off_time, pun_on_time, 60, 
                           odor_on_time, off_time, 
                           odor_on_time, off_time,
                           odor_on_time, off_time, 
                           ]
                ) # 21 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_22964\2517545031.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_22964\2517545031.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_22964\2517545031.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input

In [6]:
masks_path = r'D:\Data\DAN-trace-1CS\excitation_patterns\2_1'

# The path to the current acquisition folder is needed in order to save the sequence of projections.
current_acquisition = r'\2_2__DAN_20240709_160030' # D:\Data\g12\
protocol_name = r"/2_2"
images_folder = mirrors.xp_folder + current_acquisition


# img1 = imageio.imread(masks_path + '\\mask_1.png') # odour1
img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
img3 = imageio.imread(masks_path + '\\mask_3.png') # pain

# tot = mirrors.sum_patterns([img2, img3]) # odour 1 + pain
odpun = mirrors.sum_patterns([img1, img3]) # odour 1 + pain

start = time.time()

power = 2. # changed from 2

odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

for i in range(2):
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(60)

for i in range(19):
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time + pun_on_time + rep_dead_time)

mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(train_off_time + pun_on_time + 60)

for i in range(3):
    mirrors.project(img2, power); time.sleep(odor_on_time); mirrors.idle(); time.sleep(off_time)

# SAVE THE PROTOCOL
filenames_camera_0 = r'\SPC00_TM*_ANG000_CM0_CHN00_PH0.stack'
all_volumes = sorted(glob(images_folder + filenames_camera_0))
for num, volume in enumerate(all_volumes):
    current = os.path.getctime(volume)
    if current - start > 0:
        break

np.savez(images_folder + protocol_name,\
         TM=num, images=["2", "dark", "2", "dark", "2", "dark", 
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark",
                         "2", "dark", 
                         "2", "dark", "2", "dark",  "2", "dark",
                         ], \
                durations=[odor_on_time, off_time, 
                           odor_on_time, off_time, 
                           odor_on_time, 60, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time,
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time,
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + 60, 
                           odor_on_time, off_time, 
                           odor_on_time, off_time,
                           odor_on_time, off_time, 
                           ]
                ) # 21 mins

C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_22964\2700863253.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imageio.imread(masks_path + '\\mask_2.png') # odour2
C:\Users\Light Sheet User\AppData\Local\Temp\ipykernel_22964\2700863253.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img3 = imageio.imread(masks_path + '\\mask_3.png') # pain


DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 

DMD ON,  input voltage 2.000000 



In [2]:
on_time = 15
on_time_split = 10
odor_on_time = 15
pun_on_time = 10
off_time = 30
train_off_time = 1 # changed from 10
rep_dead_time = 20

durations = [odor_on_time, off_time, 
                           odor_on_time, off_time, 
                           odor_on_time, 60, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time,
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time,
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + rep_dead_time, 
                           odor_on_time, train_off_time + pun_on_time + 60, 
                           odor_on_time, off_time, 
                           odor_on_time, off_time,
                           odor_on_time, off_time, 
                           ]

t = 0
for i in durations:
    t+= i


print(t/60)

21.0
